# **Configuration du dataset RSCID**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_json = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/dataset_rsicd.json"
path_img_folder = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/images"
path_test_patient = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/test_patient.json"

In [ ]:
import json

# Charger le fichier JSON
with open(path_json, 'r') as file:
    data = json.load(file)

# Création du dataset
dataset_map = {}
for item in data["images"]:
    filename = item['filename']
    captions = [sent['raw'] for sent in item['sentences']]
    dataset_map[filename] = captions

# Création du dataset de test
dataset_test = []
with open(path_test_patient, 'r') as file:
    data = json.load(file)

In [ ]:
# Afficher un exemple
ex = "airport_10.jpg"
if ex in dataset_map:
    print("Image name :", ex)
    print("Labels :")
    print(dataset_map[ex])

Image name : airport_10.jpg
Labels :
['some planes are parked in an airport .', 'the airport here is full of airplanes and containers .', 'the airport here is full of airplanes and containers .', 'some planes are parked in an airport .', 'some planes are parked in an airport .']


In [ ]:
print(len(dataset_map))
print(len(data))

10921
1092


In [ ]:
# On supprime les éléments de test dans le dataset
keys_to_check = list(dataset_map.keys())
for key in keys_to_check:
    if key in data:
        del dataset_map[key]

print(len(dataset_map))

9829


In [ ]:
for i in range(len(data)):
    data[i] = data[i][:-4] + '.txt'

In [ ]:
print(data)

['airport_61.txt', 'airport_62.txt', 'airport_63.txt', 'airport_64.txt', 'airport_65.txt', 'airport_66.txt', 'airport_67.txt', 'airport_68.txt', 'airport_69.txt', 'airport_7.txt', 'airport_70.txt', 'airport_71.txt', 'airport_72.txt', 'airport_73.txt', 'airport_74.txt', 'airport_75.txt', 'airport_76.txt', 'airport_77.txt', 'airport_78.txt', 'airport_79.txt', 'airport_8.txt', 'airport_80.txt', 'airport_81.txt', 'airport_82.txt', 'airport_83.txt', 'airport_84.txt', 'airport_85.txt', 'airport_86.txt', 'airport_87.txt', 'airport_88.txt', 'airport_89.txt', 'airport_9.txt', 'airport_90.txt', 'airport_91.txt', 'airport_92.txt', 'airport_93.txt', 'bareland_71.txt', 'bareland_72.txt', 'bareland_73.txt', 'bareland_74.txt', 'bareland_75.txt', 'bareland_76.txt', 'bareland_77.txt', 'bareland_78.txt', 'bareland_79.txt', 'bareland_8.txt', 'bareland_80.txt', 'bareland_81.txt', 'bareland_82.txt', 'bareland_83.txt', 'bareland_84.txt', 'bareland_85.txt', 'bareland_86.txt', 'bareland_87.txt', 'bareland_88.

# **Analyse des données de la segmentation et calcul**

In [ ]:
import os

# Récupérez les fichiers de prédiction
folder_path = '/content/drive/MyDrive/Classroom/MSV/dataset RSICD/segmentation_unet_mobiletnet/'
pred_files = os.listdir(folder_path)
print(pred_files)

pred_files = [file for file in pred_files if file not in data]
pred_files = sorted(pred_files)
print(len(pred_files))

['viaduct_406.txt', 'viaduct_407.txt', 'viaduct_408.txt', 'viaduct_409.txt', 'viaduct_41.txt', 'viaduct_410.txt', 'viaduct_411.txt', 'viaduct_412.txt', 'viaduct_413.txt', 'viaduct_414.txt', 'viaduct_415.txt', 'viaduct_416.txt', 'viaduct_417.txt', 'viaduct_418.txt', 'viaduct_419.txt', 'viaduct_42.txt', 'viaduct_420.txt', 'viaduct_43.txt', 'viaduct_44.txt', 'viaduct_45.txt', 'viaduct_46.txt', 'viaduct_47.txt', 'viaduct_48.txt', 'viaduct_49.txt', 'viaduct_5.txt', 'viaduct_50.txt', 'viaduct_51.txt', 'viaduct_52.txt', 'viaduct_53.txt', 'viaduct_54.txt', 'viaduct_55.txt', 'viaduct_56.txt', 'viaduct_57.txt', 'viaduct_58.txt', 'viaduct_59.txt', 'viaduct_6.txt', 'viaduct_60.txt', 'viaduct_61.txt', 'viaduct_62.txt', 'viaduct_63.txt', 'viaduct_64.txt', 'viaduct_65.txt', 'viaduct_66.txt', 'viaduct_67.txt', 'viaduct_68.txt', 'viaduct_69.txt', 'viaduct_7.txt', 'viaduct_70.txt', 'viaduct_71.txt', 'viaduct_72.txt', 'viaduct_73.txt', 'viaduct_74.txt', 'viaduct_75.txt', 'viaduct_76.txt', 'viaduct_77.txt

In [ ]:
def recup_data(filename):
  values = []
  coordinates = {}
  percentages = {}

  # Lecture du fichier texte
  with open(filename, "r") as f:
    lines = f.readlines()

  for line in lines:
    parts = line.strip().split(";")
    if len(parts) == 3:
      value = int(parts[0].strip())
      percentage = float(parts[1].strip())
      coord_str = parts[2].strip()[1:-1]
      coord = tuple(map(int, coord_str.split(",")))
      values.append(value)
      percentages[value] = percentage
      coordinates[value] = coord

  return values, coordinates, percentages

In [ ]:
def eliminate_simple(values, coordinates, percentages):
  values_2 = []
  coordinates_2 = {}
  percentages_2 = {}

  # Seuil pour le pourcentage
  seuil = 5

  for label, percentage in percentages.items():
    if percentage >= seuil:
      values_2.append(label)
      coordinates_2[label] = coordinates[label]
      percentages_2[label] = percentage

  return values_2, coordinates_2, percentages_2

In [ ]:
import math

def compare_coordinates_closest(c1):
  closest_distances = {}

  for key1, coord1 in c1.items():
    closest_distance = math.inf
    closest_coord = None

    for key2, coord2 in c1.items():
      if key1 != key2:
        distance = math.sqrt((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2)
        if distance < closest_distance:
          closest_distance = distance
          closest_coord = key2
    closest_distances[key1] = closest_coord

  return closest_distances

def remove_duplicates(closest_distances):
  final_distances = {}
  reversed_distances = {v: k for k, v in closest_distances.items()}
  seen_pairs = set()

  for key, value in closest_distances.items():
    if (key, value) not in seen_pairs and (value, key) not in seen_pairs:
      final_distances[key] = value
      seen_pairs.add((key, value))
      seen_pairs.add((value, key))

  return final_distances

In [ ]:
def compare_coordinates(closest, coordinates):
  comparisons = []

  for coord, neight in closest.items():
    coord_x, coord_y = coordinates[coord]
    neighbor_x, neighbor_y = coordinates[neight]

    if coord_x < neighbor_x:
      if coord_y < neighbor_y:
        comparisons.append((coord, "front right", neight))
      elif coord_y > neighbor_y:
        comparisons.append((coord, "behind right", neight))
      else:
        comparisons.append((coord, "to the right of", neight))
    elif coord_x > neighbor_x:
      if coord_y < neighbor_y:
        comparisons.append((coord, "front left", neight))
      elif coord_y > neighbor_y:
        comparisons.append((coord, "behind left", neight))
      else:
        comparisons.append((coord, "to the left of", neight))
    else:
      if coord_y < neighbor_y:
        comparisons.append((coord, "in front", neight))
      elif coord_y > neighbor_y:
        comparisons.append((coord, "behind", neight))
      else:
        comparisons.append((coord, "on", neight))

  return comparisons

In [ ]:
import csv
import numpy as np
from PIL import Image
import re

names_class = [
    'Unlabeled',
    'Artificial Surface',
    'Natural Surface',
    'Water',
    'Vegetation',
    'Building',
    'Fences & Poles',
    'People & Animals',
    'Vehicles',
    'Markers & Obstacles'
]

print(names_class)

['Unlabeled', 'Artificial Surface', 'Natural Surface', 'Water', 'Vegetation', 'Building', 'Fences & Poles', 'People & Animals', 'Vehicles', 'Markers & Obstacles']


In [ ]:
i = 1941
for i in range(i, len(pred_files)) :
  filename_path = folder_path + pred_files[i]

  # Initialisation des listes pour stocker les valeurs
  v, c, p = recup_data(filename_path)
  v2, c2, p2 = eliminate_simple(v, c, p)
  comparisons = []

  if(len(c2) >= 2):
    closest = compare_coordinates_closest(c2)
    closest = remove_duplicates(closest)
    comparisons = compare_coordinates(closest, c2)

  # Enregistrement des données
  txt = ""
  for value in v2 :
    tmp = names_class[value]
    txt += tmp + " " + f"{p2[value]:.2f}" + "\n"

  for cl1, dir, cl2  in comparisons:
    txt += names_class[cl1] + " " + dir + " " + names_class[cl2] + "\n"

  print(txt)

  # Enregistrement dans un fichier
  pred_path_file = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/" + pred_files[i]

  with open(pred_path_file, "w") as f:
    f.write(txt)

  i += 1
  print("Save text pred - ", i, " : ", pred_path_file, "OK")

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

Save text pred -  8661  :  /content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/square_181.txt OK
Artificial Surface 95.23

Save text pred -  8662  :  /content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/square_182.txt OK
Artificial Surface 74.38
Vegetation 20.67
Artificial Surface behind left Vegetation

Save text pred -  8663  :  /content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/square_183.txt OK
Artificial Surface 92.09

Save text pred -  8664  :  /content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/square_184.txt OK
Artificial Surface 81.27
Water 11.19
Artificial Surface front left Water

Save text pred -  8665  :  /content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/square_185.txt OK
Artificial Surface 80.86
Markers & Obstacles 13.02
Artificial Surface behind right Markers & Obstacles

Save text pred -  8666  : 

# **Configuration du nouveau dataset "LEGEND / ANALYSE"**

In [ ]:
import os

# Récupérez les fichiers de prédiction
folder_path = '/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/'
txt_files = os.listdir(folder_path)
print(txt_files)
print(len(txt_files))

file_names = [file_name.replace('.txt', '.jpg') for file_name in txt_files]
print(file_names)
print(len(file_names))

['baseballfield_1.txt', 'baseballfield_100.txt', 'baseballfield_101.txt', 'baseballfield_10.txt', 'baseballfield_102.txt', 'baseballfield_103.txt', 'baseballfield_105.txt', 'baseballfield_104.txt', 'baseballfield_106.txt', 'baseballfield_107.txt', 'baseballfield_108.txt', 'baseballfield_11.txt', 'baseballfield_110.txt', 'baseballfield_109.txt', 'baseballfield_111.txt', 'baseballfield_112.txt', 'baseballfield_113.txt', 'baseballfield_114.txt', 'baseballfield_116.txt', 'baseballfield_115.txt', 'baseballfield_117.txt', 'baseballfield_118.txt', 'baseballfield_119.txt', 'baseballfield_12.txt', 'baseballfield_120.txt', 'baseballfield_122.txt', 'baseballfield_123.txt', 'baseballfield_121.txt', 'baseballfield_124.txt', 'baseballfield_125.txt', 'baseballfield_127.txt', 'baseballfield_126.txt', 'baseballfield_129.txt', 'baseballfield_128.txt', 'baseballfield_13.txt', 'baseballfield_133.txt', 'baseballfield_132.txt', 'baseballfield_131.txt', 'baseballfield_134.txt', 'baseballfield_130.txt', 'base

In [ ]:
import json
import random
import csv

data_list = []

# Chemin du fichier JSON de sortie
output_json_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.json"
output_csv_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.csv"
instruction = "Write a short description of a remote sensing image with the following information."

for tmp in file_names:
    # Récupération de l'indice
    ind = file_names.index(tmp)

    # Récupération de la description de l'image
    input_txt = ""

    with open('/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/' + txt_files[ind], "r") as file:
        input_txt = file.read()

    # Récupération de la phrase de la contexte
    output_txt = random.choice(dataset_map[tmp])

    data = {
        "input": input_txt,
        "output": output_txt,
        "instruction": instruction
    }

    data_list.append(data)

# Écriture des données dans le fichier JSON
with open(output_json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print("Fichier JSON créé avec succès :", output_json_file_path)

# Convert JSON data to CSV
csv_headers = ["input", "output", "instruction"]

with open(output_csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_headers)
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print("Fichier CSV créé avec succès :", output_csv_file_path)


Fichier JSON créé avec succès : /content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.json
Fichier CSV créé avec succès : /content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset.csv


# **Dataset de test**

In [ ]:
dir_unet_mobiletnet = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/segmentation_unet_mobiletnet"
pred_files_seg = os.listdir(dir_unet_mobiletnet)
dir_calcul_segmentation = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation"
pred_files_calcul = os.listdir(dir_calcul_segmentation)

items = [item for item in pred_files_seg if item not in pred_files_calcul]
print(items)
print(len(items))

In [ ]:
folder_path = '/content/drive/MyDrive/Classroom/MSV/dataset RSICD/segmentation_unet_mobiletnet/'

i = 0
for i in range(i, len(items)) :
  filename_path = folder_path + items[i]

  # Initialisation des listes pour stocker les valeurs
  v, c, p = recup_data(filename_path)
  v2, c2, p2 = eliminate_simple(v, c, p)
  comparisons = []

  if(len(c2) >= 2):
    closest = compare_coordinates_closest(c2)
    closest = remove_duplicates(closest)
    comparisons = compare_coordinates(closest, c2)

  # Enregistrement des données
  txt = ""
  for value in v2 :
    tmp = names_class[value]
    txt += tmp + " " + f"{p2[value]:.2f}" + "\n"

  for cl1, dir, cl2  in comparisons:
    txt += names_class[cl1] + " " + dir + " " + names_class[cl2] + "\n"

  print(txt)

  # Enregistrement dans un fichier
  pred_path_file = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/test/" + items[i]

  with open(pred_path_file, "w") as f:
    f.write(txt)

  i += 1
  print("Save text pred - ", i, " : ", pred_path_file, "OK")

In [ ]:
# Récupérez les fichiers de prédiction
folder_path = '/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/test'
txt_files = os.listdir(folder_path)
file_names = [file_name.replace('.txt', '.jpg') for file_name in txt_files]

data_list = []

# Chemin du fichier JSON de sortie
output_json_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.json"
output_csv_file_path = "/content/drive/MyDrive/Classroom/MSV/dataset RSICD/remote_sensing_dataset_test.csv"
instruction = "Write a short description of a remote sensing image with the following information."

for tmp in file_names:
    # Récupération de l'indice
    ind = file_names.index(tmp)

    # Récupération de la description de l'image
    input_txt = ""

    with open('/content/drive/MyDrive/Classroom/MSV/dataset RSICD/calcul_segmentation/test/' + txt_files[ind], "r") as file:
        input_txt = file.read()

    # Récupération de la phrase de la contexte
    output_txt = random.choice(dataset_map[tmp])

    data = {
        "input": input_txt,
        "output": output_txt,
        "instruction": instruction
    }

    data_list.append(data)

# Écriture des données dans le fichier JSON
with open(output_json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print("Fichier JSON créé avec succès :", output_json_file_path)

# Convert JSON data to CSV
csv_headers = ["input", "output", "instruction"]

with open(output_csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_headers)
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print("Fichier CSV créé avec succès :", output_csv_file_path)
